In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/honors/honors_data

/content/drive/MyDrive/honors/honor_data


### 数据处理 I
1. 去重
2. 按时间排序

In [3]:
import numpy as np
import pandas as pd

In [4]:
# 遍历每个表格
dfs = []
for i in range(22):
    file_name = f"股票超话{i}.xlsx"
    df = pd.read_excel(file_name)
    dfs.append(df)
# 合并成一个表格
data = pd.concat(dfs, ignore_index=True)

In [5]:
data.shape

(196722, 10)

In [6]:
data.isna().sum() # 判断缺失情况

昵称           1
头像           0
博文链接         0
时间           0
来自于          0
转发数      30463
评论数      26657
点赞数      15335
博文内容        17
图片链接    119573
dtype: int64

In [7]:
data = data.dropna(subset = '博文内容')

In [8]:
data.shape

(196705, 10)

In [9]:
# 导出分词前完整数据
# 按时间顺序排列
data['时间'] = pd.to_datetime(data['时间'])
sorted = data.sort_values(by='时间')
selected_columns = ["时间", "博文内容"]
new = sorted[selected_columns]
new.to_excel('data.xlsx', index=False)

### 数据处理 II
1. 预处理特定文本

In [10]:
import re

In [12]:
new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'\n', ' ', x))

<ipython-input-12-c91db221dace>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'\n', ' ', x))


In [13]:
new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'分享图片', '', x))
new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'@[^ ]+', '', x))
new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'\b\w+的微博视频\b', '', x))

<ipython-input-13-b83f78435def>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'分享图片', '', x))
<ipython-input-13-b83f78435def>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'@[^ ]+', '', x))
<ipython-input-13-b83f78435def>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [14]:
from collections import Counter

# 提起标签
column_name = "博文内容"
pattern = r"#(.*?)#"

# 初始化标签计数器
tag_counter = Counter()

# 遍历每行数据，统计标签出现次数
for index, row in new.iterrows():
    content = row[column_name]
    tags = re.findall(pattern, content)
    tag_counter.update(tags)

# 找到出现次数超过20次的标签，并按出现次数降序排序
popular_tags = [(tag, count) for tag, count in tag_counter.items() if count > 20]
popular_tags.sort(key=lambda x: x[1], reverse=True)

# 打印标签和出现次数
# for tag, count in popular_tags:
    # print(f"标签: {tag}，出现次数: {count}")

In [15]:
# 替换热门标签
for tag, _ in popular_tags:
    tg = f'#{tag}#'
    new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(tg, '', str(x)))

<ipython-input-15-9977e73106df>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(tg, '', str(x)))


In [16]:
# 去数字
new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'\d+', '', x))

<ipython-input-16-86607ff1fe03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['博文内容'] = new['博文内容'].apply(lambda x: re.sub(r'\d+', '', x))


In [17]:
new['博文内容'] = new['博文内容'].apply(lambda x: x.strip() if x.strip() != '' else None)
new.dropna(subset=['博文内容'], inplace=True)

<ipython-input-17-70a19b9abc3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['博文内容'] = new['博文内容'].apply(lambda x: x.strip() if x.strip() != '' else None)
<ipython-input-17-70a19b9abc3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.dropna(subset=['博文内容'], inplace=True)


In [18]:
new.shape

(195503, 5)

In [19]:
new.reset_index(drop=True, inplace=True)

In [20]:
# 导出预处理后完整数据
pre_data = new.copy(deep = True)
pre_data.to_excel('preprocessed_data.xlsx', index=False)

### 数据处理 III

去除其他字符

In [21]:
for index, row in pre_data.iterrows():
    content_list = row['博文内容']

    # 将列表内容合并为字符串
    content_string = ''.join(content_list)

    # 使用正则表达式匹配非中英文字母字符并替换为空字符串
    content_cleaned = re.sub(r'[^a-zA-Z\u4e00-\u9fff]', '', content_string)

    pre_data.at[index, '博文内容'] = content_cleaned

In [22]:
pre_data.dropna(subset=['博文内容'], inplace=True)
pre_data.reset_index(drop=True, inplace=True)

In [23]:
pre_data.shape

(195503, 5)

In [24]:
pre_data["内容"] = pre_data["博文内容"].copy()

In [36]:
pre_data.head(5)

,时间,博文内容,内容
0,2017-07-13 15:16:37,吃奶粉的孩子的确更容易胖因为奶粉中有数倍于母乳的脂肪和蛋白质是宝宝不需要的而且会造成体重过量...,吃奶粉的孩子的确更容易胖因为奶粉中有数倍于母乳的脂肪和蛋白质是宝宝不需要的而且会造成体重过量...
1,2017-07-14 10:00:33,这只股票怎样,这只股票怎样
2,2017-07-14 12:06:18,双横臂,双横臂
3,2017-07-14 14:46:46,主属客户证件重复客户合并,主属客户证件重复客户合并
4,2017-07-14 15:11:15,撑,撑


### 数据处理IV

分词停用词

In [37]:
# 加载停用词
with open('/content/drive/MyDrive/honors/preprocess/stoplist.txt', encoding='utf8') as f:
    lines = f.readlines()
stopwords = [i.strip() for i in lines]

In [38]:
import jieba
# 定义分词函数
def tokenize_text(text):
    words = jieba.lcut(text)
    words = [word for word in words if word not in stopwords]
    return words

# 分词
to_tokenized = pre_data.copy(deep = True)
to_tokenized['博文内容'] = to_tokenized['博文内容'].astype(str)
to_tokenized["博文内容"] = to_tokenized["博文内容"].apply(lambda x: tokenize_text(x))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.908 seconds.
DEBUG:jieba:Loading model cost 0.908 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [39]:
# 导出分词后完整数据
tokenized = to_tokenized.copy(deep = True)
tokenized.to_excel('tokenized_data.xlsx', index=False)

### 数据处理 V

处理特殊字符

In [40]:
from collections import Counter

column_name = '博文内容'

# 将文本列转换为单个单词的列表
characters = [char for text in tokenized[column_name] for char in text]

# 使用Counter统计每个单词的出现次数
char_counts = Counter(characters)

# 找到出现次数前100的标签，并按出现次数降序排序
popular_chars = char_counts.most_common(500)
# popular_char = [(char, count) for char, count in char_counts.items() if count > 20]
# popular_char.sort(key=lambda x: x[1], reverse=True)

# 打印每个单词及其出现次数
# i = 1
# for char, count in popular_chars:
#    print(f"单词{i}：{char} ，出现次数：{count} ")
#    i += 1

In [41]:
# 从文本中逐个删除指定的单词
words_to_remove = ['超话', '微博', '转发', '朋友', '网页', '链接', '发布', '头条', '文章', '股票']

removed = tokenized.copy(deep = True)
removed['博文内容'] = removed['博文内容'].apply(lambda x: [word for word in x if word not in words_to_remove])

In [42]:
for index, row in removed.iterrows():
    content_list = row['博文内容']

    if all(word == '' for word in content_list):
        removed.at[index, '博文内容'] = np.nan

In [43]:
removed.dropna(subset=['博文内容'], inplace=True)
removed.reset_index(drop=True, inplace=True)

In [44]:
removed.shape

(191460, 3)

In [45]:
# 导出分词+去空后完整数据
removed.to_excel('completed_data.xlsx', index=False)

### 导出样本数据

In [ ]:
# 导出样本
# sample_tokenized = tokenized.sample(n = 20000)
# sample_tokenized.to_excel("sample_completed_data.xlsx", index=False)

### 关键词检查

In [53]:
keywords = ['医疗', '医药', '药业', '生物', '制药', '中药', '医院']
count = 0

for row in removed["博文内容"]:
    if any(keyword in row for keyword in keywords):
        count += 1

print(count)

11581


In [54]:
keywords = ['医疗', '医药', '药业', '生物', '制药', '中药', '医院']
keyword_count = {}

for keyword in keywords:
    count = 0
    for row in removed["博文内容"]:
        if keyword in row:
            count += 1
    keyword_count[keyword] = count

print(keyword_count)

{'医疗': 3839, '医药': 5237, '药业': 2547, '生物': 2248, '制药': 1330, '中药': 1037, '医院': 215}


In [56]:
from itertools import combinations

keywords = ['医疗', '医药', '药业', '生物', '制药', '中药', '医院']
keyword_combinations = combinations(keywords, 2)
combination_count = {}

for combination in keyword_combinations:
    keyword1, keyword2 = combination
    count = 0
    for row in removed["博文内容"]:
        if keyword1 in row and keyword2 in row:
            count += 1
    combination_count[combination] = count

print(combination_count)

{('医疗', '医药'): 1098, ('医疗', '药业'): 448, ('医疗', '生物'): 568, ('医疗', '制药'): 366, ('医疗', '中药'): 218, ('医疗', '医院'): 95, ('医药', '药业'): 829, ('医药', '生物'): 759, ('医药', '制药'): 576, ('医药', '中药'): 333, ('医药', '医院'): 71, ('药业', '生物'): 470, ('药业', '制药'): 454, ('药业', '中药'): 237, ('药业', '医院'): 25, ('生物', '制药'): 268, ('生物', '中药'): 93, ('生物', '医院'): 31, ('制药', '中药'): 189, ('制药', '医院'): 23, ('中药', '医院'): 11}
